In [1]:
import pandas as pd, numpy as np

# Combining Datasets: Merge and Join

For convenience, let's redesign the `display()` function from the previous section

In [2]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)


## Categories of Joins

The `pd.merge()` function implements a number of types of joins: the one=to-one, many-to-one and many-to-many joins. all three types of joins are accessed via an identical call to the `pd.merge()` interface.

### One-to-one join

This is one of the simplest type of joins out of the three, which is in many ways similar to the column-wise concatenation studied before

In [3]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [4]:
df3=pd.merge(df1,df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


The `pd.merge()` function instantly recognizes the employee column in both the dataframes and automatically joins them using that column as the key. Notice how the order of the entries is not necessarily maintained. Also remember, the `pd.merge()` function discards the index, unless we use a separate case of merge that we'll discuss momentarily

### Many-to-one Join

These are joins when one of the two key columns contain repeating entries. In this case the resulting dataframe preserves the repitition and joins accordingly

In [5]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

### Many-to-Many joins

In instances where both the key columns contain repeating entries, then the join displays every possible way of combining the corresponding instances about all the repeating entries in both the rows. This might get clearer from the example below

In [6]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1','df5','pd.merge(df1,df5)')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


## Specification of the Merge Key

In all of the above examples, the pd.merge function automatically chooses the column about which it merges both the tables. Sometimes it's necessary that we merge two tables about a column of out own choice. This can be done as follows

### The `on` argument

In [7]:
pd.merge(df1,df2,on='employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


This option works only if both the tables contain the column name assigned to the `on` argument

### The `left_on` and `right_on` argument

Say we want to joint two different tables about columns which have different names on both the tables but have the same type of data. This can be done as follows

In [10]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
print(display('df1','df3'))
pd.merge(df1,df3,left_on='employee',right_on='name')

(df1
   employee        group
 0      Bob   Accounting
 1     Jake  Engineering
 2     Lisa  Engineering
 3      Sue           HR
 
 df3
    name  salary
 0   Bob   70000
 1  Jake   80000
 2  Lisa  120000
 3   Sue   90000,
   employee        group  name  salary
 0      Bob   Accounting   Bob   70000
 1     Jake  Engineering  Jake   80000
 2     Lisa  Engineering  Lisa  120000
 3      Sue           HR   Sue   90000)

As you can see, the resultant table has two columns that show the same column named `employee` and `name`

In [25]:
pd.merge(df1,df3,left_on='employee',right_on='name').drop(labels='name',axis='columns')

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### The `left_index` and `right_index` keywords

Say you want to merge about the index instead of a column. This can be done as follows

In [35]:
df1a=df1.set_index('employee')
df3a=df3.set_index('name')
print(df1a,df3a)
pd.merge(df1a,df3a,left_index=True,right_index=True)

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR       salary
name        
Bob    70000
Jake   80000
Lisa  120000
Sue    90000


,group,salary
employee,,
Bob,Accounting,70000
Jake,Engineering,80000
Lisa,Engineering,120000
Sue,HR,90000


Or, we can implement the `join()` method that defaults merging two tables about its indeces

In [36]:
df1a.join(df3a)

,group,salary
employee,,
Bob,Accounting,70000
Jake,Engineering,80000
Lisa,Engineering,120000
Sue,HR,90000


If we choose to join one tables index with another tables column, this can be done as follows

In [45]:
pd.merge(df1,df3a,left_on='employee',right_index=True)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


## Specifying the Arithmetic for Join

Pandas also has functions that can specifically implement left join, right join, inner join or outer join

In [47]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6','df7')

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread
,name,drink
0,Mary,wine
1,Joseph,beer


Notice below how merging the two dataframes results discarding multiple rows.

In [49]:
pd.merge(df6,df7)

,name,food,drink
0,Mary,bread,wine


This is the same as performing an inner join

In [50]:
pd.merge(df6,df7,how='inner')

,name,food,drink
0,Mary,bread,wine


Say we want to instead perform a left, right or an outer join. This can be done as such

In [51]:
pd.merge(df6,df7,how='left')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


In [52]:
pd.merge(df6,df7,how='right')

,name,food,drink
0,Mary,bread,wine
1,Joseph,NaN,beer


In [53]:
pd.merge(df6,df7,how='outer')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,beer


## Overlapping Column Names

Say two of your datasets that you're trying to join have two column names with the exact same names 

In [57]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
pd.merge(df8,df9,on='name')

,name,rank_x,rank_y
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


Notice in the previous example, how the merge function adds suffexes `_x` and `_y` to make the output columns unique.

This suffix can be explicitly specified to our likeness

In [59]:
pd.merge(df8,df9,on='name',suffixes=["_l","_r"])

,name,rank_l,rank_r
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


## Examples

In [150]:
pop=pd.read_csv('state-population.csv')
areas=pd.read_csv('state-areas.csv')
abbrevs=pd.read_csv('state-abbrevs.csv')
display('pop.head()','areas.head()','abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

Say we are tasked with ranking the US states by their 2010 population Density

In [151]:
merged=pd.merge(abbrevs,pop,left_on='abbreviation',right_on='state/region').drop('state/region',axis='columns')
merged

,state,abbreviation,ages,year,population
0,Alabama,AL,under18,2012,1117489.0
1,Alabama,AL,total,2012,4817528.0
2,Alabama,AL,under18,2010,1130966.0
3,Alabama,AL,total,2010,4785570.0
4,Alabama,AL,under18,2011,1125763.0
...,...,...,...,...,...
2443,Wyoming,WY,under18,1993,137458.0
2444,Wyoming,WY,total,1991,459260.0
2445,Wyoming,WY,under18,1991,136720.0
2446,Wyoming,WY,under18,1990,136078.0


Let's double-check whether there were any mismatches here

In [152]:
merged.isnull().any()

state           False
abbreviation    False
ages            False
year            False
population      False
dtype: bool

In [153]:
final=pd.merge(merged,areas,on='state')

In [154]:
final=pd.merge(merged,areas,on='state')
final['density']=final['population']/final['area (sq. mi)']

In [159]:
final=final[(final['year']==2010) & (final['ages']=='total')]

In [160]:
data2010=final[['state','density','abbreviation']].set_index('state').sort_values(by='state')
data2010

,density,abbreviation
state,,
Alabama,91.287603,AL
Alaska,1.087509,AK
Arizona,56.214497,AZ
Arkansas,54.948667,AR
California,228.051342,CA
Colorado,48.493718,CO
Connecticut,645.600649,CT
Delaware,460.445752,DE
District of Columbia,8898.897059,DC
